# Lab 4.1 - Introduction to Telegram

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

In this notebook, we will use the [Telegram](https://telegram.org/) API to build a chatbot that you can send and receive messages from. 

An [Application Programming Interface](https://g.co/kgs/RhQbFu) (or API) is a set of functions that allows you to interact with an application (such as Telegram). 

Telegram is a messaging app similar to Facebook messenger or WhatsApp. We use Telegram because it is an fast and encrypted messaging application that is open-source. This means that anyone can verify their code or get their hands dirty and create their own chatbots. WHat is also nice is that you can easily share your own chatbot with your friends and family or ask anybody to test it.

**Main goal of this notebook**: The most important goal of this notebook is to have your first Telegram chatbot up and running.

**At the end of this notebook, you will**:
* **Have created your own Telegram account**: 
* **Have access to the CLTL Telegram chatbot**: 
* **Have created a Telegram chatbot**: 
* **Be able to receive messages sent to a chatbot**:  
* **Be able to send messages via a chatbot**

### Getting Telegram

The first thing you will need to do is download the [Telegram](https://telegram.org/apps) application. You can choose to have it on mobile (Android or iOS) or access it via a Destop application (Windows, masOS). Using Telegram requires a phone number, so be prepared to provide this information. The set up of your account will ask for a photo (optional) and a name. You can use any name or nickname that you like. 

Telegram will feel very familiar to you if you have used any messaging app before. The interface is intuitive with regards to how to send messages and see you latest chats. 

A benefit of Telegram is that to chat with someone you do not need to share your phone number, you can simple share your username. Still, if you are sensitive about these apps, you can go to the privacy settings and disable some of the options there such as who can see your "Last seen". 

![Privacy settings](images/privacy.png)

## Chatting with the CLTL Telegram chatbot

First things first, you will need the following Python packages to run this notebook. Remember that you can install a package by using `conda install requests` from the command line.

In [1]:
import json
import datetime
import requests

### Sending messages to the chatbot

We have created a chatbot for our research group [(Computational Lexicology & Terminology Lab or CLTL](http://www.cltl.nl/)). In order to send a message to the CLTL chatbot you can click this [link](http://t.me/cltl_bot). You can also go to "New message" in the Telegram application, and then search for the username "*cltl_bot*".

You can send messages to this bot, but currently it has no capability to respond, so you will just be chatting with yourself.

### Receiving messages to the chatbot

In order to control a bot through the Telegram API you need to have a private token. This token allows you to read and process the messages sent to the chatbot, while nobody else can unless they have this token. The token for the CLTL bot has been given to you in a **JSON** file. Please place the "*tokens.json*" file under the "*data*" folder of this lab.

The folowing function will read the **JSON** file into a Python variable for usage in this notebook.

In [2]:
def read_token():
    tokens_path = './data/tokens.json'
    with open(tokens_path) as f:
        tokens = json.load(f)

    return tokens['CLTL_token']

In [4]:
### this only works if you place the tokens.json file with the right structure and your token in the data folder
CLTL_TOKEN = read_token()

Next, to filter the messages coming from a specific user you will need to know that user ID. Let's start by getting your own user ID. Not surprisingly, there is a bot for that! (You can see a pattern here)

To get your user ID you can send a message to the [userinfobot](https://t.me/userinfobot). Sending any message to this bot will return your Telegram user information, including your ID. You can take this ID and assign it to a Python variable.

In [5]:
user_id = 408043639

The last thing you will need is some methods to use the telegram API. We have done this for you. Take your time to look at the code. It is no problem if you do not get everything. You are only going to use this code and not change it.

The following Python class *BotHandler* gives you access to the three methods:

* Get all the messages sent to the bot
* Get the last message sent to the bot by a specific user
* Let the bot send a message to a specific user. 

In [8]:
class BotHandler:
    #### This is an init function that creates an instance of the class BotHandler that we are defining
    #### It assign the secret token and sets the URL for the API
    def __init__(self, token):
        self.token = token
        self.api_url = "https://api.telegram.org/bot{}/".format(token)

    def get_all_messages(self, offset=None, timeout=200):  ### If things get slow, timeout sets the limit to wait for a response
        """ Function to get all messages sent to the bot """
        method = 'getUpdates'
        params = {'timeout': timeout, 'offset': offset}
        resp = requests.get(self.api_url + method, params)
        
        return resp.json()['result']
    
    def filter_messages_by(self, update, chat_id):
        """ Function to filter messages by user id"""
        return 'message' in update.keys() and update['message']['chat']['id'] == chat_id

    def get_last_message_by(self, chat_id):
        """ Function to get the last message sent to the bot by a specific user"""
        messages = self.get_all_messages()
        messages_by_user = list(filter(lambda m: self.filter_messages_by(m, chat_id), messages))

        last_message = None
        if messages_by_user and 'message' in messages_by_user[-1].keys():
            last_message = messages_by_user[-1]['message']['text']

        return last_message
    
    def send_message_to(self, chat_id, text):
        """ Function to send a message from the bot to a specific user"""
        params = {'chat_id': chat_id, 'text': text}
        method = 'sendMessage'
        resp = requests.post(self.api_url + method, params)

Now you have everything you need for chatting with the bot. Let's build some simple functionality. For example, we can teach it to respond to greetings.

We first create an instance of the above class that we call *bot*. Above, we can see that the init function requires the token as a parameter. Assuming, you have loaded the token, we can create our bot:

In [10]:
bot = BotHandler(CLTL_TOKEN)

Now our *bot* is alive and we can do things:

In [12]:
hour = datetime.datetime.now().hour
greetings = ('hello', 'hi', 'greetings', 'hey', 'good morning', 'good afternoon')

last_message = bot.get_last_message_by(user_id)

if last_message:
    if last_message.lower() in greetings and 6 <= hour < 12:
        bot.send_message_to(user_id, 'Good Morning')

    elif last_message.lower() in greetings and 12 <= hour < 17:
        bot.send_message_to(user_id, 'Good Afternoon')

    elif last_message.lower() in greetings and 17 <= hour < 23:
        bot.send_message_to(user_id, 'Good Evening')

    else:
        bot.send_message_to(user_id, 'Good Night')
else:
     bot.send_message_to(user_id, 'Good day')

That's it! We now have a bot that can greet us.

![Bot greetings](images/greetings.png)

## Creating your own Telegram bot

Now that you have created your Telegram account, you can create your own bot following these [instructions](https://core.telegram.org/bots#3-how-do-i-create-a-bot). Basically, you will need to interact with a bot (called BotFather), which is in charge of creating and managing Telegram bots. 

If you are interested in knowing more about what Telegram bots can do you can checkout the [Bot API Manual](https://core.telegram.org/bots/api) for more information.

## End of this notebook